In [59]:
import pandas as pd 
import datetime
import csv
import os 
import requests 
import datetime as dt
import time
import calendar

# Exercise

For your exercise do the following:

1. Choose a reddit page you want to crawl
2. The following fields should be present when you crawl **(10 points)**:
    - author
    - subreddit
    - date created 
    - number of comments 
    - score
    - submission title 
    - submission description
3. After crawling, save your results to a pandas dataframe **(3 points)**. 
4. Answer the following questions **(12 points)**:
    - How many submissions were you able to gather? 300
    - Who has the most submissions? 
    - Which submission has the highest score? 
    - Which submission has the highest number of comments?
    azgx29	1596244801	1201	2	nba	[Post Game Thread] The Milwaukee Bucks ( 54-12..
    - Which day of the week has the most submissions? 
    
**Tip:** _For item#4, recall how to use the aggregation functions in `pandas` like count, value_counts, sum, etc. For getting the day of the week, look into how to get the `dayofweek` from a datetime object in `pandas`. (Hint: You may need to use `pd.to_datetime` to convert your date column...)_

In [125]:
URL = "https://api.pushshift.io/reddit/submission/search/"  #query submissions

def to_utc(date):
    #This function converts an object to UTC. This is to automate the conversion 
    #of dates instead of going to https://www.unixtimeconverter.io/ 
    return int(date.replace(tzinfo=dt.timezone.utc).timestamp())
    
def to_readable_date(timestamp):
    #This function converts the UTC format to a Year-Month-Day format 
    return dt.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d")

#Declare start and end of reddit posts to extract 
start_date = dt.datetime.strptime("2020-08-01", "%Y-%m-%d")
end_date = dt.datetime.strptime("2020-08-06", "%Y-%m-%d")

date_range = (pd.date_range(
                start_date, 
                periods=(end_date - start_date).days + 2)
              .tolist())
sort_type="score"
sort="desc"
fields=["author","subreddit","created_utc","num_comments","score","title"]
subreddit = 'nba'
url = "https://api.pushshift.io/reddit/submission/search/"
results = []

#loop through the dates 
for i, s_date in enumerate(date_range):
    #prevents us from getting an index out of range error
    if i != len(date_range)-1:
        #declare end date 
        e_date = date_range[i+1]
        #call the API
        r = requests.get(url = url, params={
            'after': to_utc(s_date),
            'before': to_utc(e_date),
            'sort_type': sort_type,
            'sort': sort,
            'subreddit': subreddit,
            'fields': fields,
            "size": 500
        })

        #add logs 
        print(f"Doing {s_date.strftime('%Y-%m-%d')} to {e_date.strftime('%Y-%m-%d')}")
        if r.status_code == 200:
            results.append(r.json()['data'])
            print("=====Done")
        else:
            print("=====Skipped")
        #so that we dont get blocked from abusing the API we call it after pausing for 1 second
        time.sleep(1)
        
df = pd.DataFrame(results)

flat_list = []
#loop through the reddit results
for sublist in results:
    #check if sublist is not empty. The reason we have empty lists is because there are days wherein there are no submissions
    if sublist is not None:
        #for each dictionary in the sublist add it to the flat list 
        for item in sublist:
            flat_list.append(item)

#pandas has a useful function called from_dict which will convert a list of dictionary objects into a dataframe
df = pd.DataFrame.from_dict(flat_list)
display(df.head())
df.to_csv("reddit_nba.csv")


Doing 2020-08-01 to 2020-08-02
=====Done
Doing 2020-08-02 to 2020-08-03
=====Done
Doing 2020-08-03 to 2020-08-04
=====Done
Doing 2020-08-04 to 2020-08-05
=====Done
Doing 2020-08-05 to 2020-08-06
=====Done
Doing 2020-08-06 to 2020-08-07
=====Done


,author,created_utc,num_comments,score,subreddit,title
0,CP3_for_MvP,1596323995,17,2,nba,[Highlight] Aaron Holiday gets his shot destro...
1,Andrewski18,1596251850,1,2,nba,Michael Beasley has been posting some concerni...
2,azgx29,1596244801,1201,2,nba,[Post Game Thread] The Milwaukee Bucks ( 54-12...
3,midnightZEROx,1596245597,21,2,nba,Did Harden's Hairline Recede?
4,fbreaker,1596244173,1087,2,nba,[Highlight] Giannis buddha palms Daniel Theis ...


In [126]:
#How many submissions were you able to gather? 
 #   - Who has the most submissions? 
  #  - Which submission has the highest score? 
   # - Which submission has the highest number of comments?
    # - Which day of the week has the most submissions? 


In [132]:
#How many submissions were you able to gather? - 600
df.count()

author          600
created_utc     600
num_comments    600
score           600
subreddit       600
title           600
date            600
day_of_week     600
dtype: int64

In [128]:
#Who has the most submissions? CP3_for_MvP             8
df['author'].count()
df['author'].nunique()
df['author'].size

df['author'].value_counts().head(10)

CP3_for_MvP             8
[deleted]               8
MessiTV__net            8
GuyCarbonneauGOAT       6
auscrisos               6
NBA_MOD                 5
Balls_of_Adamanthium    4
WeathrNinja             4
fbreaker                4
mkgandkembafan          4
Name: author, dtype: int64

In [133]:
#Which submission has the highest score?  00	fbreaker	1596406090	470	3	nba	Jaylen Brown and Jayson Tatum tonight for the:...
df.sort_values(by='score', ascending=False).head()
df.nlargest(5, 'score')

,author,created_utc,num_comments,score,subreddit,title,date,day_of_week
100,fbreaker,1596406090,470,3,nba,Jaylen Brown and Jayson Tatum tonight for the:...,2020-08-02,Sunday
101,harlem50,1596332841,3,3,nba,TJ Warren and Jimmy Butler both have the same ...,2020-08-01,Saturday
300,CountAardvark,1596580301,6,3,nba,[Highlight] Booker extinguishes the Clippers w...,2020-08-04,Tuesday
0,CP3_for_MvP,1596323995,17,2,nba,[Highlight] Aaron Holiday gets his shot destro...,2020-08-01,Saturday
1,Andrewski18,1596251850,1,2,nba,Michael Beasley has been posting some concerni...,2020-07-31,Friday


In [130]:
#Which submission has the highest number of comments?532	NBA_MOD	1596754800	1556	1	nba	GAME THREAD: Portland Trail Blazers (31-38) @ .
df.sort_values(by='num_comments', ascending=False).head()
df.nlargest(5,'num_comments')

,author,created_utc,num_comments,score,subreddit,title
532,NBA_MOD,1596754800,1556,1,nba,GAME THREAD: Portland Trail Blazers (31-38) @ ...
2,azgx29,1596244801,1201,2,nba,[Post Game Thread] The Milwaukee Bucks ( 54-12...
4,fbreaker,1596244173,1087,2,nba,[Highlight] Giannis buddha palms Daniel Theis ...
535,NBA_MOD,1596749400,1016,1,nba,GAME THREAD: Los Angeles Clippers (45-22) @ Da...
370,drunkkk_,1596580054,988,1,nba,[Post Game Thread] The Phoenix Suns (29-39) de...


In [134]:
# Which day of the week has the most submissions? Tuesday

df['date'] = df.apply(lambda x: to_readable_date(x['created_utc']), axis=1)
df['date'] = pd.to_datetime(df['date'])
df['day_of_week'] = df['date'].dt.day_name()
df['day_of_week'].value_counts().head(10)

Tuesday      120
Saturday     110
Sunday       106
Wednesday     91
Monday        79
Thursday      65
Friday        29
Name: day_of_week, dtype: int64